In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


def load_image(path):
    img = Image.open(path).convert("RGB")
    img_array = np.array(img)
    return img, img_array, img_array.tobytes()


def aes_encrypt(data, key, iv):
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return cipher.encrypt(pad(data, AES.block_size))


def aes_decrypt(ciphertext, key, iv):
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(ciphertext), AES.block_size)


def visualize_images(original, encrypted, decrypted):
    fig, axs = plt.subplots(2, 3, figsize=(18, 10))

    titles = ["Orijinal Görüntü", "Şifreli (Rastgele Piksel)", "Çözülmüş Görüntü"]
    images = [original, encrypted, decrypted]

    for i in range(3):
        axs[0, i].imshow(images[i])
        axs[0, i].set_title(titles[i])
        axs[0, i].axis("off")

    for i, img in enumerate(images):
        plot_histogram(axs[1, i], img, titles[i] + " Histogram")

    plt.tight_layout()
    plt.show()


def plot_histogram(ax, image_array, title):
    for i, color in enumerate(['r', 'g', 'b']):
        ax.hist(image_array[..., i].flatten(), bins=256, color=color, alpha=0.6, label=f"{color.upper()} Kanalı")
    ax.set_title(title)
    ax.set_xlim(0, 256)
    ax.legend()


def main():
    image_path = "ornek_goruntu.png"
    original_img, img_array, img_bytes = load_image(image_path)

    key = get_random_bytes(16)
    iv = get_random_bytes(16)

    encrypted_bytes = aes_encrypt(img_bytes, key, iv)
    decrypted_bytes = aes_decrypt(encrypted_bytes, key, iv)

    decrypted_img = np.frombuffer(decrypted_bytes, dtype=np.uint8).reshape(img_array.shape)

    encrypted_view = np.frombuffer(encrypted_bytes[:len(img_bytes)], dtype=np.uint8)
    encrypted_view = encrypted_view.reshape(img_array.shape)

    visualize_images(img_array, encrypted_view, decrypted_img)


if __name__ == "__main__":
    main()
